In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [2]:
#The Encoder network for single view 3D reconstruction is a ResNet18 pretrained
#on the ImageNet dataset with the last fully-connected layer adjusted to project
#the features to a 256 dimensional embedding, "c"
from torchvision.models.resnet import resnet18 as _resnet18


In [3]:
class ImageEncoder(nn.Module):
    def __init__(self):
        super(ImageEncoder,self).__init__()
        self.encoderModel = _resnet18(pretrained=True)
        self.fc1 = nn.Linear(1000, 256)
        self.betafc = nn.Linear(256,256)
        self.gammafc = nn.Linear(256,256)
        
    def forward(self,x):
        x = self.encoderModel(x)
        #project to 256 dimensional embedding 
        x = self.fc1(x)
        # Obtain Beta and gamma inputs into conditional batch normalization
        # QUESTION Are these split or one after the other?
        beta = self.betafc(x)
        gamma = self.gammafc(x) #? gammaLayer(beta)
        return beta,gamma

In [93]:
class Block(nn.Module):
    def __init__(self):
        super(Block,self).__init__()
        self.fc1 = nn.Linear(256,256)
        self.fc2 = nn.Linear(256,256)
        self.bn1 = nn.BatchNorm1d(256)
        self.bn2 = nn.BatchNorm1d(256)

        
    def forward(self,y):
        x = y['ex']
        gamma = y['gamma']
        beta = y['beta']
        #First apply Conditional Batch Normalization
        out = gamma*self.bn1(x) + beta
        #Then ReLU activation function
        out = F.relu(out)
        #fully connected layer
        out = self.fc1(out)
        #Second CBN layer
        out = gamma*self.bn2(out) + beta
        #RELU activation
        out = F.relu(out)
        #2nd fully connected
        out = self.fc2(out)
        #Add to the input of the ResNet Block 
        out = x + out
        
        return {'ex':out, 'beta':beta, 'gamma':gamma}

In [98]:
class OccupancyModel(nn.Module):
    def __init__(self):
        super(OccupancyModel,self).__init__()
        self.blocks = self.makeBlocks()
        self.encoder = ImageEncoder()
        self.cbn = nn.BatchNorm1d(256)
        self.fc1 = nn.Linear(3,256)
        self.fc2 = nn.Linear(256,1)
        
    def makeBlocks(self):
        blocks = []
        for _ in range(5):
            blocks.append(Block())
        return nn.Sequential(*blocks)
   
  
    def forward(self,x, img):
        gamma,beta = self.encoder(img)
        x = self.fc1(x)
        #5 pre-activation ResNet-blocks
        x = self.blocks({'gamma':gamma, 'beta':beta, 'ex':x })
        x = x['ex']
        x = gamma*self.cbn(x) + beta
        x = F.relu(x)
        x = self.fc2(x)
        x = torch.sigmoid(x)
        return x

In [99]:
model = OccupancyModel()
#Input to the occupancy network architecture is the 
#output of the encoder network and a batch of 3D coordinates. 
coords = torch.rand(64,3)
image = torch.rand(64,3,7,7)
model.eval()

p = model(coords,image)


In [27]:
#load some data:
#The .npz contains "points, occupancies, loc, scale" 
import numpy
with numpy.load("/home/andrea/Documents/GradSchool/OccupancyNetworks/occupancy_networks/data/ShapeNet/02691156/fd528602cbde6f11bbf3143b1cb6076a/points.npz") as data:
    pts = torch.tensor(data["points"], dtype=torch.float)
    occupancies = torch.tensor(data["occupancies"], dtype=torch.float)

from PIL import Image
image = numpy.array(Image.open("/home/andrea/Documents/GradSchool/OccupancyNetworks/occupancy_networks/data/ShapeNet/02691156/fd528602cbde6f11bbf3143b1cb6076a/img_choy2016/015.jpg"))
#At least for this image directory, the jpgs come in as 137,137,3
image = torch.tensor(image,dtype=torch.float).permute(2,0,1)
image = image.view(1,3,137,137)

train_loader = torch.utils.data.DataLoader((image,pts,occupancies))


In [9]:
beta,gamma = encoder(image)
prob = model(sampleCoords, beta,gamma)

In [100]:
def train(epoch, model, encoder, trainloader, optimizer):
    modelCriterion = nn.BCELoss()
    model.train()
    for batch_idx, (image, pts, occupancies) in enumerate(train_loader):
        optimizer.zero_grad()
        output = model(pts, image) #a probability for each point 
        loss = modelCrition(output, occupancies)
        loss.backward()
        optimizer.step()
        #encoderLoss.backward() / a different optimizer.step()
        if batch_idx % 10 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset), 100. * batch_idx / len(train_loader),
                loss.item()))

In [101]:
optimizer = optim.Adam(model.parameters(), lr = 0.0001)
train(1,model,encoder,train_loader,optimizer)

ValueError: not enough values to unpack (expected 3, got 1)

In [31]:
#Next step: trying to figure out how to organize the data. 
#Also open question- Why are the values in the occupancies file not just 0s and 1s?
#Actually, I want to figure out how to write a proper config file and just use
#the existing scripts to start up the training if possible 
for x in trainer_loader:
    print(x.size())

torch.Size([1, 1, 3, 137, 137])
torch.Size([1, 100000, 3])
torch.Size([1, 12500])
